# Projet de reconnaissance faciale

# Reconnaissance faciale avec TensorFlow

## Preparation des données

In [9]:
#import tensorflow as tf
#from tensorflow.keras.preprocessing import image_dataset_from_directory
#
## Charger le dataset
#dataset_path = "dataset"
#batch_size = 32
#img_height = 224
#img_width = 224
#
#train_dataset = image_dataset_from_directory(
#    dataset_path,
#    validation_split=0.2,
#    subset="training",
#    seed=123,
#    image_size=(img_height, img_width),
#    batch_size=batch_size
#)
#
#val_dataset = image_dataset_from_directory(
#    dataset_path,
#    validation_split=0.2,
#    subset="validation",
#    seed=123,
#    image_size=(img_height, img_width),
#    batch_size=batch_size
#)
#
## Récupérer les noms de classes avant la transformation
#class_names = train_dataset.class_names
#print("Class names:", class_names)
#
## Normaliser les images
#normalization_layer = tf.keras.layers.Rescaling(1./255)
#
## Appliquer la normalisation
#train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
#val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

In [10]:
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Paramètres
dataset_path = "dataset"
batch_size = 32
img_height = 224
img_width = 224

# Fonction pour organiser les images par angles avec diagnostic
def organize_images_by_angles(class_path):
    angles = {
        'face': [],
        'left': [],
        'right': [],
        'up': [],
        'down': []
    }

    for img_file in os.listdir(class_path):
        # Extraire le numéro de l'image
        try:
            # Utiliser une expression régulière plus flexible
            match = re.search(r'face_(\d+)', img_file)
            if match:
                num = int(match.group(1))
                full_path = os.path.join(class_path, img_file)

                if 0 <= num <= 4:
                    angles['face'].append(full_path)
                elif 5 <= num <= 9:
                    angles['left'].append(full_path)
                elif 10 <= num <= 14:
                    angles['right'].append(full_path)
                elif 15 <= num <= 19:
                    angles['up'].append(full_path)
                elif 20 <= num <= 24:
                    angles['down'].append(full_path)
        except (ValueError, TypeError):
            # Ignorer les fichiers qui ne correspondent pas au motif
            continue

    return angles

# Récupérer les noms de classes
class_names = os.listdir(dataset_path)
class_names = [nom for nom in class_names if os.path.isdir(os.path.join(dataset_path, nom))]

# Préparation des données
images = []
labels = []
image_paths = []

for classe, nom in enumerate(class_names):
    class_path = os.path.join(dataset_path, nom)

    # Organiser les images par angles
    angles_images = organize_images_by_angles(class_path)

    # Vérifier que chaque catégorie a des images
    val_images = []
    for angle in ['face', 'left', 'right', 'up', 'down']:
        if angles_images[angle]:
            val_images.append(np.random.choice(angles_images[angle], 1)[0])
        else:
            print(f"ATTENTION : Pas d'image pour l'angle {angle} dans {nom}")

    # Vérifier qu'on a bien des images de validation
    if not val_images:
        print(f"ERREUR : Aucune image de validation pour {nom}")
        continue  # Passer à la classe suivante

    # Charger les images de validation
    for val_path in val_images:
        try:
            img = image.load_img(val_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)

            images.append(img_array)
            labels.append(classe)
            image_paths.append(val_path)
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {val_path}: {e}")

    # Charger les images d'entraînement (les autres)
    train_images = [
        path for angle_list in angles_images.values() for path in angle_list
        if path not in val_images
    ]

    for train_path in train_images:
        try:
            img = image.load_img(train_path, target_size=(img_height, img_width))
            img_array = image.img_to_array(img)

            images.append(img_array)
            labels.append(classe)
            image_paths.append(train_path)
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {train_path}: {e}")

# Convertir en numpy arrays
images = np.array(images)
labels = np.array(labels)
image_paths = np.array(image_paths)

# Normalisation
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Création des datasets TensorFlow
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
train_dataset = (train_dataset
    .batch(batch_size)
    .map(lambda x, y: (normalization_layer(x), y))
    .prefetch(tf.data.AUTOTUNE)
)

# Créer le dataset de validation en prenant 1 image sur 6 (les images de validation)
val_images = images[::6]
val_labels = labels[::6]

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = (val_dataset
    .batch(batch_size)
    .map(lambda x, y: (normalization_layer(x), y))
    .prefetch(tf.data.AUTOTUNE)
)

# Afficher les fichiers de validation
print("\nFichiers de validation :")
for classe in range(len(class_names)):
    print(f"\nClasse {class_names[classe]} :")
    classe_val_paths = image_paths[(labels == classe) & (np.array([path in image_paths[labels == classe][0:5] for path in image_paths]))]
    for path in classe_val_paths:
        print(os.path.basename(path))

# Autres informations de distribution
print("\nNoms des classes:", class_names)
print("Nombre total d'images:", len(images))
print("Nombre d'images par classe:", len(images) // len(class_names))
print("\nNombre d'images d'entraînement:", len(train_dataset) * batch_size)
print("Nombre d'images de validation:", len(val_dataset) * batch_size)


Fichiers de validation :

Classe Edwin_Treny :
face_3.jpg
face_8.jpg
face_10.jpg
face_16.jpg
face_22.jpg

Classe Franck_Tene :
face_4.jpg
face_8.jpg
face_14.jpg
face_18.jpg
face_23.jpg

Classe Mathis_Wauters :
face_0.jpg
face_7.jpg
face_14.jpg
face_19.jpg
face_23.jpg

Classe Samuel_Jarjanette :
face_1.jpg
face_6.jpg
face_13.jpg
face_16.jpg
face_24.jpg

Noms des classes: ['Edwin_Treny', 'Franck_Tene', 'Mathis_Wauters', 'Samuel_Jarjanette']
Nombre total d'images: 100
Nombre d'images par classe: 25

Nombre d'images d'entraînement: 128
Nombre d'images de validation: 32


## Définir le modèle

In [11]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

# Charger MobileNetV2 sans la dernière couche
base_model = MobileNetV2(input_shape=(img_height, img_width, 3),
                         include_top=False,
                         weights='imagenet')

# Geler les couches du modèle pré-entraîné
base_model.trainable = False

# Ajouter des couches personnalisées
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')  # Utiliser class_names
])

# Compiler le modèle
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Résumé du modèle
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Entraîner le modèle

In [12]:
epochs = 10

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs
)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 796ms/step - accuracy: 0.3182 - loss: 2.5096 - val_accuracy: 0.8235 - val_loss: 0.7154
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 415ms/step - accuracy: 0.7014 - loss: 0.7877 - val_accuracy: 0.4706 - val_loss: 1.0222
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 379ms/step - accuracy: 0.4963 - loss: 1.3433 - val_accuracy: 0.8824 - val_loss: 0.4670
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 378ms/step - accuracy: 0.7346 - loss: 0.6590 - val_accuracy: 1.0000 - val_loss: 0.1290
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 374ms/step - accuracy: 0.9479 - loss: 0.2409 - val_accuracy: 1.0000 - val_loss: 0.0798
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 392ms/step - accuracy: 0.9939 - loss: 0.1551 - val_accuracy: 1.0000 - val_loss: 0.0592
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 393ms/step - accuracy: 0.9661 - loss: 0.1345 - val_accuracy: 1.0000 - val_loss: 0.0386
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 392ms/step - accuracy: 0.9847 - loss: 0.0716 - val_accuracy: 1.0000 - val_loss:

## Évaluer le modèle et ajuster les hyperparamètres

In [13]:
# Tester le modèle sur des données de validation
val_loss, val_acc = model.evaluate(val_dataset)
print(f"Validation accuracy: {val_acc:.2f}")

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    ])

# Appliquer la data augmentation
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x), y))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 259ms/step - accuracy: 1.0000 - loss: 0.0198
Validation accuracy: 1.00
Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 510ms/step - accuracy: 0.7101 - loss: 0.7156 - val_accuracy: 1.0000 - val_loss: 0.0148
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 476ms/step - accuracy: 0.8893 - loss: 0.2966 - val_accuracy: 1.0000 - val_loss: 0.0374
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 485ms/step - accuracy: 0.9360 - loss: 0.3307 - val_accuracy: 1.0000 - val_loss: 0.0260
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 479ms/step - accuracy: 0.9327 - loss: 0.2054 - val_accuracy: 1.0000 - val_loss: 0.0102
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 478ms/step - accuracy: 0.8765 - loss: 0.2585 - val_accuracy: 1.0000 - val_loss: 0.0087
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 520ms/step - accuracy: 0.9630 - loss: 0.1422 - val_accuracy: 1.0000 - val_loss: 0.0098
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 543ms/step - accuracy: 0.9661 - loss: 0.1317 - val_accuracy: 1.0000 - val_loss: 0.0114
Epoch 8/10
4/4 ━━━━

## Sauvegarder le modèle

In [14]:
model.save("face_recognition_model.keras")


## Reconnaissance faciale en temps réel

In [15]:
import cv2
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import img_to_array
import os
import tkinter as tk
from tkinter import Tk, Label
from tkinter import simpledialog
from tkinter import messagebox

# Charger le modèle sauvegardé
model = tf.keras.models.load_model("face_recognition_model.keras")

# Initialiser le classificateur de visage d'OpenCV
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Dimensions des images attendues
img_height = 224
img_width = 224

# Fonction pour effectuer une prédiction
def predict_face(face, model, class_names):
    face = cv2.resize(face, (img_width, img_height))  # Redimensionner
    face_array = img_to_array(face) / 255.0  # Normaliser
    face_array = np.expand_dims(face_array, axis=0)  # Ajouter une dimension batch
    predictions = model.predict(face_array)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)
    return predicted_class, confidence

def show_message(message, duration=5):
    """
    Affiche un message dans une fenêtre pendant une durée spécifiée.
    """
    root = Tk()
    root.title("Prochaine étape")
    root.geometry("300x100")
    label = Label(root, text=message, font=("Helvetica", 16), wraplength=280)
    label.pack(expand=True)
    root.after(duration * 1000, root.destroy)  # Ferme la fenêtre après `duration` secondes
    root.mainloop()

def capture_face(prenom, nom):
    # Créer le dossier pour enregistrer les images si il n'existe pas
    folder_name = f"dataset/{prenom}_{nom}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    # Ouvrir la caméra
    cap = cv2.VideoCapture(0)
    captured_images = 0
    directions = ["Regardez a gauche", "Regardez a droite", "Regardez vers le haut", "Regardez vers le bas", "Regardez droit devant"]
    images_per_direction = 5  # Nombre d'images par direction
    total_images = len(directions) * images_per_direction

    direction_index = 0

    while captured_images < total_images:
        # Afficher le texte de l'étape actuelle
        current_direction = directions[direction_index]
        print(f"Etape : {current_direction}")

        # Pause de 3 secondes avec affichage de l'étape actuelle
        for i in range(3, 0, -1):
            ret, frame = cap.read()
            if not ret:
                break
            # Ajouter le texte pour guider l'utilisateur
            cv2.putText(frame, f"{current_direction} dans {i} secondes",
                        (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            cv2.imshow('Capturer le visage', frame)
            cv2.waitKey(1000)

        # Capturer les 50 images pour cette étape
        while captured_images < (direction_index + 1) * images_per_direction:
            ret, frame = cap.read()
            if not ret:
                break

            # Convertir en niveaux de gris pour la détection
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Détection des visages
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

            for (x, y, w, h) in faces:
                # Extraire le visage
                face = frame[y:y+h, x:x+w]

                # Enregistrer l'image
                face_filename = f"{folder_name}/face_{captured_images}.jpg"
                cv2.imwrite(face_filename, face)
                captured_images += 1

                # Ajouter un rectangle et des informations à l'écran
                cv2.putText(frame, f"Image {captured_images}/{total_images}",
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

            # Afficher l'étape actuelle pendant la capture
            cv2.putText(frame, f"Etape actuelle : {current_direction}",
                        (10, frame.shape[0] - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

            # Afficher le flux vidéo avec les visages détectés
            cv2.imshow('Capturer le visage', frame)

            # Quitter avec la touche 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cap.release()
                cv2.destroyAllWindows()
                return

        # Passer à la prochaine direction
        direction_index = (direction_index + 1) % len(directions)

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()
    show_message(f"Enregistrement terminé. Les visages ont été enregistrés dans le dossier : {folder_name}")

# Fonction pour analyser les visages
def analyze_face():
    cap = cv2.VideoCapture(0)

    # Dictionnaire pour stocker les données des visages détectés
    face_data = {}  # Clé : ID de visage, Valeur : {classe, score de confiance}

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convertir en niveaux de gris pour la détection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Détection des visages
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Mettre à jour les données des visages
        new_face_data = {}
        for i, (x, y, w, h) in enumerate(faces):
            face_id = f"face_{i}"  # ID unique pour chaque visage détecté
            face = frame[y:y+h, x:x+w]

            # Prédire la classe et la confiance pour le visage actuel
            predicted_class, confidence = predict_face(face, model, class_names)

            # Si le visage est nouveau ou si la confiance est meilleure, mettre à jour
            if face_id not in face_data or confidence > face_data[face_id]['confidence']:
                new_face_data[face_id] = {'class': predicted_class, 'confidence': confidence}
            else:
                # Sinon, conserver les données précédentes
                new_face_data[face_id] = face_data[face_id]

            # Dessiner un rectangle autour du visage et ajouter le texte
            color = (0, 255, 0)  # Vert pour le rectangle
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            text = f"{new_face_data[face_id]['class']} ({new_face_data[face_id]['confidence']:.2f})"
            cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        # Mettre à jour les données des visages
        face_data = new_face_data

        # Afficher le flux vidéo avec les prédictions
        cv2.imshow('Reconnaissance Faciale', frame)

        # Quitter avec la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    cv2.destroyAllWindows()


# Interface graphique avec Tkinter
def create_interface():
    # Fenêtre principale
    window = tk.Tk()
    window.title("Reconnaissance Faciale")

    # Fonction pour enregistrer un visage
    def start_capture():
        prenom = simpledialog.askstring("Prénom", "Entrez votre prénom:")
        nom = simpledialog.askstring("Nom", "Entrez votre nom:")
        if prenom and nom:
            capture_face(prenom, nom)

    # Fonction pour analyser les visages
    def start_analyze():
        analyze_face()

    # Boutons pour chaque fonctionnalité
    capture_button = tk.Button(window, text="Enregistrer un visage", command=start_capture)
    capture_button.pack(pady=10)

    analyze_button = tk.Button(window, text="Analyser un visage", command=start_analyze)
    analyze_button.pack(pady=10)

    # Lancer l'interface
    window.mainloop()

# Lancer l'interface graphique
create_interface()
